# Churn Prediction using ANN 

In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score
import keras
from keras.models import Sequential
from keras.layers import Dense

In [4]:
data=pd.read_csv('Churn_Modelling.csv')

In [5]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
data.shape

(10000, 14)

In [8]:
data.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [14]:
#selecting features
X=data.iloc[:,3:13].values
y=data.iloc[:,-1].values

In [15]:
X

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [16]:
y

array([1, 0, 1, ..., 1, 1, 0], dtype=int64)

In [17]:
data.corr()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
RowNumber,1.000000,0.004202,0.005840,0.000783,-0.006495,-0.009067,0.007246,0.000599,0.012044,-0.005988,-0.016571
CustomerId,0.004202,1.000000,0.005308,0.009497,-0.014883,-0.012419,0.016972,-0.014025,0.001665,0.015271,-0.006248
CreditScore,0.005840,0.005308,1.000000,-0.003965,0.000842,0.006268,0.012238,-0.005458,0.025651,-0.001384,-0.027094
Age,0.000783,0.009497,-0.003965,1.000000,-0.009997,0.028308,-0.030680,-0.011721,0.085472,-0.007201,0.285323
Tenure,-0.006495,-0.014883,0.000842,-0.009997,1.000000,-0.012254,0.013444,0.022583,-0.028362,0.007784,-0.014001
Balance,-0.009067,-0.012419,0.006268,0.028308,-0.012254,1.000000,-0.304180,-0.014858,-0.010084,0.012797,0.118533
NumOfProducts,0.007246,0.016972,0.012238,-0.030680,0.013444,-0.304180,1.000000,0.003183,0.009612,0.014204,-0.047820
HasCrCard,0.000599,-0.014025,-0.005458,-0.011721,0.022583,-0.014858,0.003183,1.000000,-0.011866,-0.009933,-0.007138
IsActiveMember,0.012044,0.001665,0.025651,0.085472,-0.028362,-0.010084,0.009612,-0.011866,1.000000,-0.011421,-0.156128
EstimatedSalary,-0.005988,0.015271,-0.001384,-0.007201,0.007784,0.012797,0.014204,-0.009933,-0.011421,1.000000,0.012097


In [23]:
#Encoding the dataset
label_encoder_x1=LabelEncoder()
X[:,1]=label_encoder_x1.fit_transform(X[:,1])
label_encoder_x2=LabelEncoder()
X[:,2]=label_encoder_x2.fit_transform(X[:,2])

transformer=ColumnTransformer(transformers=[('Geography' , OneHotEncoder() , [1])], remainder='passthrough')
X=transformer.fit_transform(X.tolist())
X=X.astype('float64')

In [28]:
#splitting the dataset into training and test model
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [29]:
X_train[0].shape

(12,)

In [30]:
#Feature Scaling
ss=StandardScaler()
X_train=ss.fit_transform(X_train)
X_test=ss.fit_transform(X_test)

In [33]:
#creating model
cls=Sequential()

cls.add(Dense(output_dim=6, init='uniform', activation='relu', input_dim=12))
cls.add(Dense(output_dim=6, init='uniform', activation='relu'))
cls.add(Dense(output_dim=1, init='uniform', activation='sigmoid'))

#compile the model
cls.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

C:\Users\Sakshi Gupta\anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=12, units=6, kernel_initializer="uniform")`
  after removing the cwd from sys.path.
C:\Users\Sakshi Gupta\anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=6, kernel_initializer="uniform")`
  """
C:\Users\Sakshi Gupta\anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1, kernel_initializer="uniform")`
  


In [34]:
cls.fit(X_train,y_train,batch_size=20,epochs=100)


Epoch 1/100
8000/8000 [==============================] - 2s 252us/step - loss: 0.5542 - accuracy: 0.7929
Epoch 2/100
8000/8000 [==============================] - 1s 75us/step - loss: 0.4091 - accuracy: 0.8223
Epoch 3/100
8000/8000 [==============================] - 1s 74us/step - loss: 0.3744 - accuracy: 0.8491
Epoch 4/100
8000/8000 [==============================] - 1s 72us/step - loss: 0.3592 - accuracy: 0.8531
Epoch 5/100
8000/8000 [==============================] - 1s 73us/step - loss: 0.3541 - accuracy: 0.8556
Epoch 6/100
8000/8000 [==============================] - 1s 69us/step - loss: 0.3506 - accuracy: 0.8583
Epoch 7/100
8000/8000 [==============================] - 1s 74us/step - loss: 0.3478 - accuracy: 0.8594
Epoch 8/100
8000/8000 [==============================] - 1s 74us/step - loss: 0.3462 - accuracy: 0.8587
Epoch 9/100
8000/8000 [==============================] - 1s 70us/step - loss: 0.3452 - accuracy: 0.8600
Epoch 10/100
8000/8000 [==============================] - 1s 72

In [35]:
y_pred=cls.predict(X_test)

In [36]:
y_pred=(y_pred>0.5)

In [37]:
y_pred

array([[False],
       [False],
       [False],
       ...,
       [ True],
       [False],
       [False]])

In [38]:
y_pred.shape

(2000, 1)

In [39]:
y_test=(y_test>0.5)

In [40]:
y_test

array([False, False, False, ...,  True,  True,  True])

In [41]:
y_test.shape

(2000,)

In [42]:
cm=confusion_matrix(y_pred,y_test)
cm

array([[1550,  221],
       [  57,  172]], dtype=int64)

In [44]:
accuracy=accuracy_score(y_pred,y_test)

In [46]:
print('The accuracy of the model is ',accuracy)

The accuracy of the model is  0.861
